spatial error model: interpret coefficient the same as a standard linear OLS. spatial lag or combo model: cannot do so, due to diffusion/spillover effects.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pysal.lib import weights
from pysal.model import spreg
from scipy.stats.mstats import zscore
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tools.tools import add_constant

shp_path = 'data/tracts_shapefile/tracts_shapefile.shp'
indicators_path = 'data/tracts_indicators_grades_eras_index.csv'
response = 'vehicles_per_household'

spat_diag = False

In [2]:
df = pd.read_csv(indicators_path, dtype={'geoid':str, 'state':str, 'county':str})
df.shape

(72663, 154)

In [3]:
gdf = gpd.read_file(shp_path).set_index('GEOID')
gdf.shape

(74133, 12)

In [4]:
# restrict modeling to only urban tracts
df = df[df['is_urban'] == 1]
df.shape

(46311, 154)

In [5]:
df.corr()[response].abs().sort_values(ascending=False).head(10)

vehicles_per_household    1.000000
prop_single_fam           0.728190
vehicles_per_capita       0.714770
med_rooms_per_home        0.664678
prop_drive_alone          0.662129
available_vehicles        0.545621
pop_density               0.519471
med_hh_income             0.515185
prop_deadend              0.510276
prop_1939_earlier         0.496005
Name: vehicles_per_household, dtype: float64

## Modeling

In [6]:
df['pop_density'] = df['pop_density'] / 1000 #put these regression coefficients into units of 1000s of persons

In [7]:
# identify the era dummies in the dataframe
era_primary_dummies = [c for c in df.columns if 'dummy_primary_' in c and '_1939_earlier' not in c]

# get the state dummies
states = df['state_abbrev'].unique()
state_dummies = sorted([s for s in states if s != 'CA']) #all but CA
len(state_dummies)

50

In [8]:
%%time
# create county dummies
df['st_county'] = df['state'].astype(str) + df['county'].astype(str)
counties = df['st_county'].unique()
for county in counties:
    df[county] = df['st_county'].map(lambda x: 1 if x==county else 0)

county_dummies = counties[1:].tolist()
#county_dummies = sorted([c for c in counties if c != '06037']) #all but LA county
print(len(county_dummies))

1390
Wall time: 18.6 s


In [9]:
# define which dummies to use as the spatial fixed effects
# if including both county + state, you'll get colinearity unless you drop one county from each state?
fixed_effects = county_dummies #+ state_dummies
len(fixed_effects)

1390

In [10]:
def get_response_and_design(df, response, predictors, condition_number=True):
    
    # select predictors and drop any rows with nulls in the response or predictors
    df_model = df.replace([np.inf, -np.inf], np.nan).dropna(subset=predictors + [response])

    # create design matrix and response vector (and response as matrix for pysal)
    X = df_model[predictors]
    y = df_model[response]

    # drop columns that are constants (to prevent perfect colinearity)
    # this happens if a county has no observations, for instance
    X = X.loc[:, X.nunique() != 1]
    
    # what are the geoids of the observations retained in the response vector + design matrix?
    geoids = df_model['geoid'].values
    
    if condition_number:
        cn = np.linalg.cond(zscore(X))
        return y, X, geoids, cn
    else:
        return y, X, geoids

In [11]:
def make_pysal_table(model, precision=4, ignore=None):
    
    try:
        idx = model.name_z
    except:
        idx = model.name_x
    
    z_stat = np.array(model.z_stat)
    table = pd.DataFrame({'beta' : model.betas.flatten(),
                          's.e.' : model.std_err,
                          'z'    : z_stat[:, 0],
                          'p'    : z_stat[:, 1]}, 
                          index=idx)
    
    if ignore is not None:
        to_drop = [c for c in ignore if c in table.index]
        table = table.drop(to_drop, axis='rows')
    
    return table.round(precision)

## Model 1

grid index + spatial fixed effects

In [12]:
%%time
regressors1 = ['grid_index_geom']
predictors1 = regressors1 + fixed_effects
y, X, geoids, cn = get_response_and_design(df, response, predictors1)
print(cn)

122.42852150811426
Wall time: 17.9 s


In [13]:
%%time
# estimate the model with OLS
result1 = sm.OLS(y, add_constant(X)).fit()
print(result1.rsquared)

C:\Anaconda\envs\ox\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


0.5285580352119494
Wall time: 8.02 s


In [14]:
%%time
# calculate spatial weights matrix for spatially-explicit alternative specification
W1 = weights.Queen.from_dataframe(gdf.loc[geoids], silence_warnings=True)
W1.transform = 'r'

Wall time: 35.5 s


In [15]:
%%time
# first check ols diagnostics to see nature of spatial dependence
Y = pd.DataFrame(y)
if spat_diag:
    ols = spreg.ols.OLS(y=Y.values, x=X.values, w=W1, spat_diag=True, moran=True)
    print(ols.moran_res)
    print(ols.rlm_lag, ols.rlm_error)

Wall time: 972 µs


In [16]:
%%time
sp_er_model1 = spreg.GM_Error_Het(y=Y.values, x=X.values, w=W1, name_w='W1',
                                  name_x=X.columns.tolist(), name_y=response)
table1 = make_pysal_table(sp_er_model1, ignore=fixed_effects)
print(sp_er_model1.pr2)
print(table1)

0.5089320846122712
                   beta    s.e.         z    p
CONSTANT         2.0903  0.1543   13.5492  0.0
grid_index_geom -0.4866  0.0111  -43.8225  0.0
lambda           0.6519  0.0040  162.5834  0.0
Wall time: 6.97 s


## Model 2

grid index + additional controls + spatial fixed effects

In [17]:
%%time
regressors2 = ['grid_index_geom', #griddedness
               'pop_density', 'prop_single_fam', 'med_rooms_per_home', 'mean_household_size', #settlement density/scale
               'med_hh_income', 'mean_commute_time', #economic (income and job proximity)
               'intersect_density', 'length_mean', #street spatial scale
               'grade_mean'] #hilliness
predictors2 = regressors2 + fixed_effects
y, X, geoids, cn = get_response_and_design(df, response, predictors2)
print(cn)

185.91636928812554
Wall time: 18.2 s


In [18]:
%%time
# estimate the model with OLS
result2 = sm.OLS(y, add_constant(X)).fit()
print(result2.rsquared)

0.8575453840345233
Wall time: 9.1 s


In [19]:
%%time
# calculate spatial weights matrix for spatially-explicit alternative specification
W2 = weights.Queen.from_dataframe(gdf.loc[geoids], silence_warnings=True)
W2.transform = 'r'

Wall time: 32 s


In [20]:
%%time
# check ols diagnostics to see nature of spatial dependence
Y = pd.DataFrame(y)
if spat_diag:
    ols = spreg.ols.OLS(y=Y.values, x=X.values, w=W2, spat_diag=True, moran=True)
    print(ols.moran_res)
    print(ols.rlm_lag, ols.rlm_error)

Wall time: 0 ns


In [30]:
%%time
sp_er_model2 = spreg.GM_Error_Het(y=Y.values, x=X.values, w=W2, name_w='W2',
                                  name_x=X.columns.tolist(), name_y=response)
table2 = make_pysal_table(sp_er_model2, ignore=fixed_effects)
print(sp_er_model2.pr2)
print(table2)

0.8527514505017132
                       beta    s.e.         z       p
CONSTANT             0.8729  0.1049    8.3192  0.0000
grid_index_geom     -0.1849  0.0069  -26.8037  0.0000
pop_density         -0.0052  0.0003  -16.0073  0.0000
prop_single_fam      0.4671  0.0065   71.7585  0.0000
med_rooms_per_home   0.0256  0.0018   14.5975  0.0000
mean_household_size  0.1788  0.0030   59.8594  0.0000
med_hh_income        0.0000  0.0000   57.4592  0.0000
mean_commute_time   -0.0033  0.0002  -13.1017  0.0000
intersect_density   -0.0006  0.0000  -12.5258  0.0000
length_mean          0.0003  0.0000    7.9205  0.0000
grade_mean          -0.2297  0.1152   -1.9939  0.0462
lambda               0.6249  0.0044  140.8927  0.0000
Wall time: 7.18 s


In [36]:
%%time
# re-estimate as standardized regression
sp_er_model2_std = spreg.GM_Error_Het(y=zscore(Y), x=zscore(X), w=W2, name_w='W2',
                                      name_x=X.columns.tolist(), name_y=response)
table2_std = make_pysal_table(sp_er_model2_std, ignore=fixed_effects)
print(sp_er_model2_std.pr2)
print(table2_std)

0.8527514505081624
                       beta    s.e.         z       p
CONSTANT             0.0091  0.0038    2.3776  0.0174
grid_index_geom     -0.0839  0.0031  -26.8037  0.0000
pop_density         -0.0615  0.0038  -16.0073  0.0000
prop_single_fam      0.3024  0.0042   71.7585  0.0000
med_rooms_per_home   0.0693  0.0048   14.5975  0.0000
mean_household_size  0.2271  0.0038   59.8594  0.0000
med_hh_income        0.2883  0.0050   57.4592  0.0000
mean_commute_time   -0.0535  0.0041  -13.1017  0.0000
intersect_density   -0.0431  0.0034  -12.5258  0.0000
length_mean          0.0229  0.0029    7.9205  0.0000
grade_mean          -0.0065  0.0033   -1.9939  0.0462
lambda               0.6249  0.0044  140.8927  0.0000
Wall time: 7.93 s


## Model 3

grid index components + additional controls + spatial fixed effects

In [37]:
%%time
regressors3 = ['orientation_order', 'prop_4way', 'straightness', #grid index components
               'pop_density', 'prop_single_fam', 'med_rooms_per_home', 'mean_household_size', #settlement density/scale
               'med_hh_income', 'mean_commute_time', #economic (income and job proximity)
               'intersect_density', 'length_mean', #street spatial scale
               'grade_mean'] #hilliness
predictors3 = regressors3 + fixed_effects
y, X, geoids, cn = get_response_and_design(df, response, predictors3)
print(cn)

201.76756966497868
Wall time: 19.1 s


In [38]:
%%time
# estimate the model with OLS
result3 = sm.OLS(y, add_constant(X)).fit()
print(result3.rsquared)

0.8584870270799135
Wall time: 8.7 s


In [39]:
%%time
# calculate spatial weights matrix for spatially-explicit alternative specification
W3 = weights.Queen.from_dataframe(gdf.loc[geoids], silence_warnings=True)
W3.transform = 'r'

Wall time: 30.7 s


In [40]:
%%time
# check ols diagnostics to see nature of spatial dependence
Y = pd.DataFrame(y)
if spat_diag:
    ols = spreg.ols.OLS(y=Y.values, x=X.values, w=W3, spat_diag=True, moran=True)
    print(ols.moran_res)
    print(ols.rlm_lag, ols.rlm_error)

Wall time: 0 ns


In [41]:
%%time
sp_er_model3 = spreg.GM_Error_Het(y=Y.values, x=X.values, w=W3, name_w='W3',
                                  name_x=X.columns.tolist(), name_y=response)
table3 = make_pysal_table(sp_er_model3, ignore=fixed_effects)
print(sp_er_model3.pr2)
print(table3)

0.8537053051991855
                       beta    s.e.         z       p
CONSTANT             1.0457  0.1076    9.7194  0.0000
orientation_order   -0.0396  0.0045   -8.7782  0.0000
prop_4way           -0.1247  0.0072  -17.2857  0.0000
straightness        -0.2212  0.0239   -9.2520  0.0000
pop_density         -0.0050  0.0003  -15.4118  0.0000
prop_single_fam      0.4708  0.0065   72.6479  0.0000
med_rooms_per_home   0.0259  0.0018   14.7630  0.0000
mean_household_size  0.1785  0.0030   59.8627  0.0000
med_hh_income        0.0000  0.0000   57.1977  0.0000
mean_commute_time   -0.0033  0.0002  -13.1061  0.0000
intersect_density   -0.0006  0.0000  -12.5372  0.0000
length_mean          0.0003  0.0000    7.6694  0.0000
grade_mean          -0.2296  0.1146   -2.0033  0.0451
lambda               0.6211  0.0045  139.5360  0.0000
Wall time: 6.13 s


In [42]:
%%time
# re-estimate as standardized regression
sp_er_model3_std = spreg.GM_Error_Het(y=zscore(Y), x=zscore(X), w=W3, name_w='W3',
                                      name_x=X.columns.tolist(), name_y=response)
table3_std = make_pysal_table(sp_er_model3_std, ignore=fixed_effects)
print(sp_er_model3_std.pr2)
print(table3_std)

0.8537053052000249
                       beta    s.e.         z       p
CONSTANT             0.0088  0.0038    2.3128  0.0207
orientation_order   -0.0270  0.0031   -8.7782  0.0000
prop_4way           -0.0525  0.0030  -17.2857  0.0000
straightness        -0.0224  0.0024   -9.2520  0.0000
pop_density         -0.0589  0.0038  -15.4118  0.0000
prop_single_fam      0.3048  0.0042   72.6479  0.0000
med_rooms_per_home   0.0701  0.0047   14.7630  0.0000
mean_household_size  0.2267  0.0038   59.8627  0.0000
med_hh_income        0.2883  0.0050   57.1977  0.0000
mean_commute_time   -0.0535  0.0041  -13.1061  0.0000
intersect_density   -0.0427  0.0034  -12.5372  0.0000
length_mean          0.0218  0.0028    7.6694  0.0000
grade_mean          -0.0065  0.0033   -2.0033  0.0451
lambda               0.6211  0.0045  139.5360  0.0000
Wall time: 6.98 s


## Results table

In [43]:
print(sp_er_model2.n, sp_er_model2.pr2)
print(sp_er_model3.n, sp_er_model3.pr2)

45557 0.8527514505017132
45557 0.8537053051991855


In [44]:
# spatially explicit estimates
def str_format(x):
    if pd.isnull(x):
        return ' '
    elif np.abs(x) < 0.0001:
        return '<0.0001'
    else:
        return f'{x:0.4f}'
    
regressors = pd.Series(['CONSTANT'] + regressors2 + regressors3 + ['lambda']).drop_duplicates(keep='first').tolist()
table = pd.merge(left=table2, right=table3, left_index=True, right_index=True, how='outer').reindex(regressors)
table = table.applymap(str_format)
table.to_csv('data/table2.csv', index=True)
table

,beta_x,s.e._x,z_x,p_x,beta_y,s.e._y,z_y,p_y
CONSTANT,0.8729,0.1049,8.3192,<0.0001,1.0457,0.1076,9.7194,<0.0001
grid_index_geom,-0.1849,0.0069,-26.8037,<0.0001,,,,
pop_density,-0.0052,0.0003,-16.0073,<0.0001,-0.0050,0.0003,-15.4118,<0.0001
prop_single_fam,0.4671,0.0065,71.7585,<0.0001,0.4708,0.0065,72.6479,<0.0001
med_rooms_per_home,0.0256,0.0018,14.5975,<0.0001,0.0259,0.0018,14.7630,<0.0001
mean_household_size,0.1788,0.0030,59.8594,<0.0001,0.1785,0.0030,59.8627,<0.0001
med_hh_income,<0.0001,<0.0001,57.4592,<0.0001,<0.0001,<0.0001,57.1977,<0.0001
mean_commute_time,-0.0033,0.0002,-13.1017,<0.0001,-0.0033,0.0002,-13.1061,<0.0001
intersect_density,-0.0006,<0.0001,-12.5258,<0.0001,-0.0006,<0.0001,-12.5372,<0.0001
length_mean,0.0003,<0.0001,7.9205,<0.0001,0.0003,<0.0001,7.6694,<0.0001


In [45]:
# OLS estimates
results = [result1, result2, result3]
regressors = pd.Series(['const'] + regressors1 + regressors2 + regressors3).drop_duplicates(keep='first').tolist()
for result in results:
    print(round(result.rsquared, 4))
summary_col(results=results, regressor_order=regressors, drop_omitted=True, stars=True)

0.5286
0.8575
0.8585


,vehicles_per_household I,vehicles_per_household II,vehicles_per_household III
const,2.2493***,0.9253***,1.2641***
,(0.1386),(0.0768),(0.0803)
grid_index_geom,-0.8573***,-0.3070***,
,(0.0095),(0.0063),
pop_density,,-0.0079***,-0.0075***
,,(0.0003),(0.0003)
prop_single_fam,,0.4921***,0.4974***
,,(0.0054),(0.0054)
med_rooms_per_home,,0.0212***,0.0221***
,,(0.0016),(0.0016)
